In [1]:
# Deliverable 1: Retrieve Weather Data
# -------------------------------------------------------------------
# Generate a set of 2,000 random latitudes and longitudes, retrieve the nearest city, and perform an API call 
# with the OpenWeatherMap. In addition to the city weather data you gathered in this module, use your API skills 
# to retrieve the current weather description for each city. Then, create a new DataFrame containing the updated 
# weather data.


import numpy as np

from citipy import citipy

from config_weather import weather_api_key 

import requests

import time

import pandas as pd

In [2]:
#Create a new set of 2,000 random latitudes and longitudes.

lat = np.random.uniform(-90,90, 2000)
lng = np.random.uniform(-180,180, 2000)

lat_lng = zip(lat,lng) #pairs the array

coordinates = list(lat_lng) #unzipping zipped tuple into a list

coordinates

[(31.22329783790923, -31.063895105680274),
 (9.018232894579825, 29.996410808916863),
 (80.6907710516902, 146.91488378057687),
 (-89.65027584897821, 108.43622718454856),
 (30.92559811531551, -161.01910263016777),
 (-58.73012561921897, -32.89710822289737),
 (75.50746796449971, -124.63672066264621),
 (-37.18644870339769, 120.3582876295444),
 (81.31614875661307, -82.4258216928895),
 (87.43699049863673, -116.21716060856414),
 (-14.67130116276202, -21.24942665325014),
 (71.4869872117431, 157.50709270041506),
 (-22.910607780220346, -104.94681368425397),
 (26.038676949660925, -97.6048283362807),
 (31.71473736767966, 56.77827158535766),
 (-24.663175337446106, -30.615599297545145),
 (11.071823705293582, -173.75025556738473),
 (-72.46100780245925, -14.62408871348103),
 (31.31570630827818, 110.08525118384898),
 (-15.980763980468666, -163.9584610387183),
 (43.97027300347486, 143.82531441354507),
 (9.678594933976498, -64.71476240046613),
 (-12.508290609131521, 151.7330136543094),
 (-23.9890170189662

In [3]:
#Get the nearest city using the citipy module.

cities = []

for row in coordinates:
    city = citipy.nearest_city(row[0], row[1]).city_name #row[0] is lat, #row[1] is long
    if city not in cities:
        cities.append(city)
    
cities


['ribeira grande',
 'bentiu',
 'chokurdakh',
 'albany',
 'kapaa',
 'mar del plata',
 'tuktoyaktuk',
 'qaanaaq',
 'norman wells',
 'georgetown',
 'cherskiy',
 'puerto ayora',
 'san benito',
 'ravar',
 'vila velha',
 'cape town',
 'enshi',
 'alofi',
 'kitami',
 'anaco',
 'samarai',
 'altonia',
 'saskylakh',
 'rikitea',
 'klaksvik',
 'mogadishu',
 'ushuaia',
 'channel-port aux basques',
 'vaini',
 'victoria point',
 'lompoc',
 'attawapiskat',
 'hoi an',
 'rocha',
 'dikson',
 'codrington',
 'hermanus',
 'atuona',
 'busselton',
 'victoria',
 'saint-leu',
 'hobart',
 'scottsburgh',
 'luderitz',
 'ahome',
 'barentsburg',
 'thompson',
 'vardo',
 'shahrud',
 'cockburn town',
 'zyryanskoye',
 'iracoubo',
 'sitio novo do tocantins',
 'itarema',
 'mocuba',
 'poum',
 'havoysund',
 'touros',
 'guanica',
 'hailar',
 'hamilton',
 'butaritari',
 'sao joao da barra',
 'pisco',
 'lavrentiya',
 'ladushkin',
 'lokachi',
 'byron bay',
 'yellowknife',
 'mataura',
 'pevek',
 'taolanaro',
 'marienburg',
 'illo

In [4]:
# Perform an API call with the OpenWeatherMap.



#empty list to hold weather data
city_data = []

#print beginning of logging
print("Beginning of retrieval    ")
print("----------------------------")

#create counters
record_count = 1
set_count = 1

#loop through all cities
for i, city in enumerate(cities): # this lets us have both index and value #range(len(cities)) gives just index, for city in cities gives just city:
    #go through cities 50 at a time for logging purposes
    if (i % 50 == 0 and i >= 50): 
        set_count +=1
        record_count = 1 
        time.sleep(60) #tells program to stop for 60 seconds because API only lets us have 60 calls per minute on the free version to prevent time out errors
    #new city url for each city
    city_url = "https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid=" + weather_api_key + "&units=imperial" + "&q=" + city.replace(" ","+") #some city names have spaces, so we're taking them out and concatinating the names for the URL (otherwise it will find data for just the first part of the city name)
    print (f"processing record {record_count} of set {set_count} | {city}")
    #add 1 to record count
    record_count +=1

    #run API request on all the cities:
    try: 
        # Parse the JSON and retrieve data.
        weather_request=requests.get(city_url).json()
        
        #Parse out the needed data
        country = weather_request["sys"]["country"]
        latitude = weather_request["coord"]["lon"]
        longitude = weather_request["coord"]["lat"]
        max_temp = weather_request["main"]["temp_max"]
        humidity = weather_request["main"]["humidity"]
        cloudiness = weather_request["clouds"]["all"]
        wind_speed = weather_request["wind"]["speed"]
        weather_description = weather_request["weather"][0]["description"] #weather is a list of dictionaries, so look at the first row in the list (at index 0), and then the description key, with the "clear sky" value
        
        #adding data to city data list:
        city_data.append({ #appending data as a list of dictionaries
            "city":city.title(), #.title makes the first letter in each string capitalized
            "country":country,
            "latitude":latitude,
            "longitude":longitude,
            "max_temp":max_temp,
            "humidity":humidity,
            "cloudiness":cloudiness,
            "wind_speed":wind_speed,
            "weather_description":weather_description})
    #if an error comes up, skip the city
    except:
        print("City not found. Skipping...")
        pass #tells it that if city is not found, don't add previous city's data #pass is a general purpose statement to handle all errors encountered and to allow the program to continue
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    


Beginning of retrieval    
----------------------------
processing record 1 of set 1 | ribeira grande
processing record 2 of set 1 | bentiu
processing record 3 of set 1 | chokurdakh
processing record 4 of set 1 | albany
processing record 5 of set 1 | kapaa
processing record 6 of set 1 | mar del plata
processing record 7 of set 1 | tuktoyaktuk
processing record 8 of set 1 | qaanaaq
processing record 9 of set 1 | norman wells
processing record 10 of set 1 | georgetown
processing record 11 of set 1 | cherskiy
processing record 12 of set 1 | puerto ayora
processing record 13 of set 1 | san benito
processing record 14 of set 1 | ravar
processing record 15 of set 1 | vila velha
processing record 16 of set 1 | cape town
processing record 17 of set 1 | enshi
processing record 18 of set 1 | alofi
processing record 19 of set 1 | kitami
processing record 20 of set 1 | anaco
processing record 21 of set 1 | samarai
processing record 22 of set 1 | altonia
City not found. Skipping...
processing recor

processing record 39 of set 4 | rawah
processing record 40 of set 4 | ulaanbaatar
processing record 41 of set 4 | viedma
processing record 42 of set 4 | aguimes
processing record 43 of set 4 | gurgan
City not found. Skipping...
processing record 44 of set 4 | kalemie
processing record 45 of set 4 | presidencia roque saenz pena
processing record 46 of set 4 | erenhot
processing record 47 of set 4 | souillac
processing record 48 of set 4 | nizhneyansk
City not found. Skipping...
processing record 49 of set 4 | labuhan
processing record 50 of set 4 | malakal
processing record 1 of set 5 | sur
processing record 2 of set 5 | atar
processing record 3 of set 5 | pitea
processing record 4 of set 5 | oussouye
processing record 5 of set 5 | spasskoye
processing record 6 of set 5 | nelson bay
processing record 7 of set 5 | kieta
processing record 8 of set 5 | ilo
processing record 9 of set 5 | payo
processing record 10 of set 5 | ilulissat
processing record 11 of set 5 | hithadhoo
processing reco

City not found. Skipping...
processing record 25 of set 8 | laguna
processing record 26 of set 8 | kindu
processing record 27 of set 8 | belmonte
processing record 28 of set 8 | mount gambier
processing record 29 of set 8 | timra
processing record 30 of set 8 | richards bay
processing record 31 of set 8 | kaeo
processing record 32 of set 8 | eureka
processing record 33 of set 8 | shaoguan
processing record 34 of set 8 | port hardy
processing record 35 of set 8 | kendari
processing record 36 of set 8 | chara
processing record 37 of set 8 | berbera
processing record 38 of set 8 | krasnooktyabrskiy
processing record 39 of set 8 | bubaque
processing record 40 of set 8 | arlit
processing record 41 of set 8 | palasa
processing record 42 of set 8 | longyearbyen
processing record 43 of set 8 | faya
processing record 44 of set 8 | avera
processing record 45 of set 8 | bonavista
processing record 46 of set 8 | toftir
City not found. Skipping...
processing record 47 of set 8 | coquimbo
processing

processing record 14 of set 12 | gilbues
processing record 15 of set 12 | havelock
processing record 16 of set 12 | necochea
processing record 17 of set 12 | gao
processing record 18 of set 12 | broome
processing record 19 of set 12 | kobojango
City not found. Skipping...
processing record 20 of set 12 | mount isa
processing record 21 of set 12 | ambodifototra
City not found. Skipping...
processing record 22 of set 12 | sinkat
City not found. Skipping...
processing record 23 of set 12 | nacala
processing record 24 of set 12 | matay
processing record 25 of set 12 | juegang
processing record 26 of set 12 | taizhou
processing record 27 of set 12 | ust-tsilma
processing record 28 of set 12 | ocean city
processing record 29 of set 12 | jagannathpur
processing record 30 of set 12 | the valley
processing record 31 of set 12 | banjarmasin
processing record 32 of set 12 | vostok
processing record 33 of set 12 | tres arroyos
processing record 34 of set 12 | alta floresta
processing record 35 of 

processing record 49 of set 15 | bolivar
processing record 50 of set 15 | calabozo
processing record 1 of set 16 | nouadhibou
processing record 2 of set 16 | leh
processing record 3 of set 16 | columbus
processing record 4 of set 16 | south venice
processing record 5 of set 16 | mehran
processing record 6 of set 16 | bonthe
processing record 7 of set 16 | aswan
processing record 8 of set 16 | bozeman
processing record 9 of set 16 | mecca
processing record 10 of set 16 | kamenskoye
City not found. Skipping...
processing record 11 of set 16 | jinchang
processing record 12 of set 16 | moroni
processing record 13 of set 16 | cacapava do sul
processing record 14 of set 16 | kovdor
processing record 15 of set 16 | anori
processing record 16 of set 16 | asau
processing record 17 of set 16 | preobrazheniye
processing record 18 of set 16 | toliary
City not found. Skipping...
processing record 19 of set 16 | abashiri
-----------------------------
Data Retrieval Complete      
-------------------

In [5]:
city_data

[{'city': 'Ribeira Grande',
  'country': 'PT',
  'latitude': -28.7,
  'longitude': 38.5167,
  'max_temp': 66.6,
  'humidity': 94,
  'cloudiness': 75,
  'wind_speed': 14.97,
  'weather_description': 'broken clouds'},
 {'city': 'Bentiu',
  'country': 'SS',
  'latitude': 29.8333,
  'longitude': 9.2333,
  'max_temp': 76.17,
  'humidity': 78,
  'cloudiness': 96,
  'wind_speed': 11.79,
  'weather_description': 'moderate rain'},
 {'city': 'Chokurdakh',
  'country': 'RU',
  'latitude': 147.9167,
  'longitude': 70.6333,
  'max_temp': 33.08,
  'humidity': 75,
  'cloudiness': 5,
  'wind_speed': 12.39,
  'weather_description': 'clear sky'},
 {'city': 'Albany',
  'country': 'US',
  'latitude': -73.9662,
  'longitude': 42.6001,
  'max_temp': 75.52,
  'humidity': 55,
  'cloudiness': 100,
  'wind_speed': 12.3,
  'weather_description': 'overcast clouds'},
 {'city': 'Kapaa',
  'country': 'US',
  'latitude': -159.319,
  'longitude': 22.0752,
  'max_temp': 72.12,
  'humidity': 95,
  'cloudiness': 0,
  'wi

In [6]:
#add weather to a new dataframe
city_df = pd.DataFrame(city_data)

city_df


,city,country,latitude,longitude,max_temp,humidity,cloudiness,wind_speed,weather_description
0,Ribeira Grande,PT,-28.7000,38.5167,66.60,94,75,14.97,broken clouds
1,Bentiu,SS,29.8333,9.2333,76.17,78,96,11.79,moderate rain
2,Chokurdakh,RU,147.9167,70.6333,33.08,75,5,12.39,clear sky
3,Albany,US,-73.9662,42.6001,75.52,55,100,12.30,overcast clouds
4,Kapaa,US,-159.3190,22.0752,72.12,95,0,11.50,clear sky
...,...,...,...,...,...,...,...,...,...
698,Kovdor,RU,30.4758,67.5662,54.72,37,56,6.40,broken clouds
699,Anori,BR,-61.6442,-3.7728,87.49,58,64,2.33,broken clouds
700,Asau,RO,26.4000,46.4333,76.12,70,73,3.13,broken clouds
701,Preobrazheniye,RU,133.9064,42.9019,47.73,80,44,3.02,scattered clouds


In [13]:
#Export the DataFrame as WeatherPy_Database.csv into the Weather_Database folder

file_name="WeatherPy_Database.csv"

city_df.to_csv(file_name, index_label="city_id")